In [1]:
%%capture
%pip install redis rich tqdm

In [2]:
import redis
from rich.pretty import pprint

In [3]:
# creating cluster
# make sure jupyter server is connected to redis network
# `docker network connect redis_default jupyter-jupyter-1`
r = redis.RedisCluster(host='master', port=6379)

In [4]:
# set two String Keys
r.set("foo","bar")
r.set("hello","world")

True

In [5]:
# retrieve keys
print("foo:",r.get("foo"),"bar:",r.get("hello"))

foo: b'bar' bar: b'world'


In [6]:
# retrieve keys' slots
print("foo:",r.keyslot("foo"),"bar:",r.keyslot("hello"))

foo: 12182 bar: 866


In [10]:
# get cluster sharding
pprint(r.cluster_shards())

[
│   {
│   │   'slots': [(5461, 10922)],
│   │   'nodes': [
│   │   │   {
│   │   │   │   b'id': b'05515c16bbac76cf253b0e70cd27ebdc09b79dd3',
│   │   │   │   b'port': 6379,
│   │   │   │   b'ip': b'192.168.224.8',
│   │   │   │   b'endpoint': b'192.168.224.8',
│   │   │   │   b'role': b'master',
│   │   │   │   b'replication-offset': 28,
│   │   │   │   b'health': b'online'
│   │   │   },
│   │   │   {
│   │   │   │   b'id': b'e91a6710eafb8e81108b6d5825a20d310d60fa59',
│   │   │   │   b'port': 6379,
│   │   │   │   b'ip': b'192.168.224.2',
│   │   │   │   b'endpoint': b'192.168.224.2',
│   │   │   │   b'role': b'replica',
│   │   │   │   b'replication-offset': 28,
│   │   │   │   b'health': b'online'
│   │   │   }
│   │   ]
│   },
│   {
│   │   'slots': [(10923, 16383)],
│   │   'nodes': [
│   │   │   {
│   │   │   │   b'id': b'436fb0cbabe7f049e5c417908ed4778038ed1ef1',
│   │   │   │   b'port': 6379,
│   │   │   │   b'ip': b'192.168.224.4',
│   │   │   │   b'endpoint': b'192.168.224.4',
│   │   │   │   b'role': b'master',
│   │   │   │   b'replication-offset': 29,
│   │   │   │   b'health': b'online'
│   │   │   },
│   │   │   {
│   │   │   │   b'id': b'10943a0deae297563982a8b38cae4fc53a5f7ec7',
│   │   │   │   b'port': 6379,
│   │   │   │   b'ip': b'192.168.224.5',
│   │   │   │   b'endpoint': b'192.168.224.5',
│   │   │   │   b'role': b'replica',
│   │   │   │   b'replication-offset': 29,
│   │   │   │   b'health': b'online'
│   │   │   }
│   │   ]
│   },
│   {
│   │   'slots': [(0, 5460)],
│   │   'nodes': [
│   │   │   {
│   │   │   │   b'id': b'766a01466170647ab9eeb117e5b01b58a598128b',
│   │   │   │   b'port': 6379,
│   │   │   │   b'ip': b'192.168.224.6',
│   │   │   │   b'endpoint': b'192.168.224.6',
│   │   │   │   b'role': b'master',
│   │   │   │   b'replication-offset': 86,
│   │   │   │   b'health': b'online'
│   │   │   },
│   │   │   {
│   │   │   │   b'id': b'6f0f229ec2ff3e2f1ed082703fdaaee15520ddcb',
│   │   │   │   b'port': 6379,
│   │   │   │   b'ip': b'192.168.224.7',
│   │   │   │   b'endpoint': b'192.168.224.7',
│   │   │   │   b'role': b'replica',
│   │   │   │   b'replication-offset': 86,
│   │   │   │   b'health': b'online'
│   │   │   }
│   │   ]
│   }
]

In [11]:
# cleaner sharding list
import socket
shards = r.cluster_shards()
nodes = []
for shard in shards:
    for node in shard["nodes"]:
        try:
            hostname = socket.gethostbyaddr(node[b"ip"])[0]
        except:
            hostname = node[b"ip"]
        nodes.append(
            {
             "node": hostname,
             "role": node[b"role"],
             "health": node[b"health"],
             "replication-offset": node[b'replication-offset'],
             "shards": shard['slots']
            }
        )
pprint(nodes)

[
│   {
│   │   'node': 'redis-master-2.redis_default',
│   │   'role': b'master',
│   │   'health': b'online',
│   │   'replication-offset': 28,
│   │   'shards': [(5461, 10922)]
│   },
│   {
│   │   'node': 'redis-replica-2.redis_default',
│   │   'role': b'replica',
│   │   'health': b'online',
│   │   'replication-offset': 28,
│   │   'shards': [(5461, 10922)]
│   },
│   {
│   │   'node': 'redis-master-3.redis_default',
│   │   'role': b'master',
│   │   'health': b'online',
│   │   'replication-offset': 29,
│   │   'shards': [(10923, 16383)]
│   },
│   {
│   │   'node': 'redis-replica-3.redis_default',
│   │   'role': b'replica',
│   │   'health': b'online',
│   │   'replication-offset': 29,
│   │   'shards': [(10923, 16383)]
│   },
│   {
│   │   'node': 'redis-master-1.redis_default',
│   │   'role': b'master',
│   │   'health': b'online',
│   │   'replication-offset': 86,
│   │   'shards': [(0, 5460)]
│   },
│   {
│   │   'node': 'redis-replica-1.redis_default',
│   │   'role': b'replica',
│   │   'health': b'online',
│   │   'replication-offset': 86,
│   │   'shards': [(0, 5460)]
│   }
]

In [9]:
# retrieve keys' slots
print("foo:",r.keyslot("foo"),"bar:",r.keyslot("hello"))

foo: 12182 bar: 866


In [12]:
# "foo" -> redis-master-3/ redis-replica-3
# docker stop redis-master-3
r.get("foo")

b'bar'

In [13]:
# cleaner sharding list
import socket
shards = r.cluster_shards()
nodes = []
for shard in shards:
    for node in shard["nodes"]:
        try:
            hostname = socket.gethostbyaddr(node[b"ip"])[0]
        except:
            hostname = node[b"ip"]
        nodes.append(
            {
             "node": hostname,
             "role": node[b"role"],
             "health": node[b"health"],
             "replication-offset": node[b'replication-offset'],
             "shards": shard['slots']
            }
        )
pprint(nodes)

[
│   {
│   │   'node': 'redis-master-2.redis_default',
│   │   'role': b'master',
│   │   'health': b'online',
│   │   'replication-offset': 84,
│   │   'shards': [(5461, 10922)]
│   },
│   {
│   │   'node': 'redis-replica-2.redis_default',
│   │   'role': b'replica',
│   │   'health': b'online',
│   │   'replication-offset': 84,
│   │   'shards': [(5461, 10922)]
│   },
│   {
│   │   'node': b'192.168.224.4',
│   │   'role': b'master',
│   │   'health': b'fail',
│   │   'replication-offset': 71,
│   │   'shards': [(10923, 16383)]
│   },
│   {
│   │   'node': 'redis-replica-3.redis_default',
│   │   'role': b'master',
│   │   'health': b'online',
│   │   'replication-offset': 71,
│   │   'shards': [(10923, 16383)]
│   },
│   {
│   │   'node': 'redis-master-1.redis_default',
│   │   'role': b'master',
│   │   'health': b'online',
│   │   'replication-offset': 142,
│   │   'shards': [(0, 5460)]
│   },
│   {
│   │   'node': 'redis-replica-1.redis_default',
│   │   'role': b'replica',
│   │   'health': b'online',
│   │   'replication-offset': 142,
│   │   'shards': [(0, 5460)]
│   }
]